## 1) Load environment variables + initialize Groq client
Loads `GROQ_API_KEY` from `.env` and creates a Groq client instance for API calls.

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from groq import Groq


dotenv_path = find_dotenv(usecwd=True)
load_dotenv(dotenv_path=dotenv_path, override=True)

api_key = os.getenv("GROQ_API_KEY")


if not api_key:
    raise ValueError("GROQ_API_KEY not found. Put it in your .env file.")

client = Groq(api_key=api_key)

model="llama-3.1-8b-instant",
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "hello"},
    ],
temperature=0.2,

# print(resp.choices[0].message.content)


## 2) (Optional) Ollama local LLM test (commented)
Example setup for running a local Ollama model. Currently disabled.

In [62]:

# from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     model="llama3.2:3b",
#     temperature=0.2,
#     num_predict=80,   # cap output tokens (big speedup)
# )
# resp = llm.invoke("Tell me a very short one-line joke about programmers.")
# print(resp.content)

## 3) Load PDF document (data ingestion)
Loads `mlschool.pdf` using `PyPDFLoader` and creates one LangChain `Document` per page.

In [215]:
import os
from langchain_community.document_loaders import PyPDFLoader

pdf_path = os.path.abspath("pdfs/mlschool.pdf")
print("Loading:", pdf_path)

loader = PyPDFLoader(pdf_path)

# return one Document per page
pages = loader.load()
pages

print(f"Number of pages: {len(pages)}")
print("First page preview:", pages[0].page_content[:300])

Loading: /Users/harshvardhan/Documents/Placements/Projects/Knowledge Base Search LLM/pdfs/mlschool.pdf
Number of pages: 14
First page preview: Learn to Build AI & Machine Learning
Systems That Don't Suck
A live, hands-on program that will help you become an order of magnitude
better at building world-class AI/ML systems.
This program is for developers looking to solve real-
world problems using AI/ML.
Most courses are boring, too academic,


## 4) Create the RAG prompt template
Defines the instruction template used to answer questions using retrieved document context.

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
Answer the question based on the context below. If the question can't
be answered based on the context, say "I don't know".

context : {context}
Question : {question}
"""

prompt = PromptTemplate.from_template(template)
print(
    prompt.format(
        context="LangChain is a framework for developing applications powered by language models.",
        question="What is LangChain?",
    )
)


Answer the question based on the context below. If the question can't
be answered based on the context, say "I don't know".

context : LangChain is a framework for developing applications powered by language models.
Question : What is LangChain?



## 5) Quick prompt → LLM chain sanity check
Tests the prompt + model pipeline on a small, manual context string to confirm the LLM call works.

In [204]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()
print(chain.invoke({"context": "the name i was given harsh vardhan", "question": "What is my name?"}))

Your name is Harsh Vardhan.


## 6) Create embeddings + build vector store (indexing)
Embeds the PDF pages using a sentence-transformer model and stores vectors for similarity search.

In [201]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings,
)

## 7) Create retriever + test retrieval
Wraps the vector store as a retriever and verifies that relevant pages are being returned for a query.

In [184]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
docs = retriever.invoke("mlschool")
print(docs)
print("Retrieved:", len(docs))

[Document(metadata={'producer': 'Skia/PDF m143', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36', 'creationdate': '2025-12-19T05:53:56+00:00', 'title': "Building AI/ML Systems That Don't Suck", 'moddate': '2025-12-19T05:53:56+00:00', 'source': '/Users/harshvardhan/Documents/Placements/Projects/Knowledge Base Search LLM/mlschool.pdf', 'total_pages': 14, 'page': 10, 'page_label': '11'}, page_content='Live sessions take place every Monday and Thursday. Office\nhours take place on Wednesdays. Every session is recorded. You\ncan attend live or watch the recorded version later.\nHere are the upcoming cohorts:\nCohort 21:February 2-February 19, 2026.10:00 AM EST\nCohort 22:May 4-May 21, 2026.2:00 PM EDT\nYou don\'t have to wait for a specific cohort to join the program.\nYou have lifetime access, so you can join any time and lock in the\ncurrent price. The sooner you join, the cheaper it will be.\n"This is one 

## 8) Build the full RAG chain (Retriever → Prompt → Groq LLM)
Creates the end-to-end pipeline: retrieve context from PDF, format prompt, call Groq LLM, return plain text.

In [ ]:
from langchain_groq import ChatGroq
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

# 1. Correctly define the LLM object
# model = ChatGroq(
#     groq_api_key=api_key,
#     model_name="llama-3.1-8b-instant",
#     temperature=0.2
# )

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke({
    "question": "What is Machine Learning?"
}))

I don't know. The context provided does not explicitly define what Machine Learning is. However, it does mention Machine Learning in various contexts, such as "real-world AI and Machine Learning engineering skills", "building production-ready AI/ML systems", and "model-centric and data-centric AI".


## 9) Batch questions loop
Runs multiple questions through the RAG chain and prints each question and its answer.

In [210]:
question = [
    # "what is this project about?"
    # "What services does Unthinkable Solutions provide?"
    # "What technologies or tech stack do they specialize in?"
    # "Where is the company located?"
    # "Can i use Fronted technologies like Reactjs with their services?"
    # ""
    # "what is the purpose of this course?"
    # "How many modules are there in this course?"
    # "How many hours of content are there in this course?"
    # "Is there a program certificate provided upon completion?"
    # "What is the overview of this course?"

    "What is Machine Learning?"
    # "What are the prerequisites for enrolling in this course?"
    # # "What Programing languages are covered in this course?"
    "How much does this course cost?"
]

for q in question:
    print("Question:", q)
    print("Answer:", chain.invoke({
        "question": q
    }))
    print()

Question: What is Machine Learning?How much does this course cost?
Answer: Based on the provided context, I can answer the following questions:

1. What is Machine Learning? 
Machine Learning is not explicitly defined in the provided context. However, it is mentioned as a key topic in the course "Building AI/ML Systems That Don't Suck". 

2. How much does this course cost?
The course costs $500, with a discounted price of $300.

